In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import re
import seaborn as sns
pd.set_option('mode.chained_assignment', None)


In [2]:
df = pd.read_csv("full_data_02_21.csv")

In [3]:
df = df.iloc[:,1:]  ## ignoring double index

## Starting to handle the data

##### 1. Getting rid of ' , '  

In [4]:
df['Common Dolphin'] = df['Common Dolphin'].str.replace(',', '')    ## 1,000 --> 1000

##### 2. Replacing single string values with the number 1

In [5]:
df['Common Dolphin'][df['Common Dolphin']=='Com.'] = '1'
df['Common Dolphin'][df['Common Dolphin']=='Com'] = '1'
df['Common Dolphin'][df['Common Dolphin']=='Common'] = '1' 
df['Common Dolphin'][df['Common Dolphin']=='Common’s'] = '1'
df['Pacific White sided Dolphin'][df['Pacific White sided Dolphin']=='Pacific'] = '1'
df['Pacific White sided Dolphin'][df['Pacific White sided Dolphin']=='Pacific White-sided'] = '1'

df['Rissos Dolphin'][df['Rissos Dolphin']=='Rissos'] = '1'
df['Rissos Dolphin'][df['Rissos Dolphin']=="Risso's"] = '1'


##### 3. Replacing \n values with spaces

In [6]:
df.replace(r'\n', ' ', regex=True, inplace = True)  ## '\n' in the df --> ' '


#### 4. Extracting numbers from string values

In [7]:
## A function that returns a column with numbers only

def extracting_numbers(df,col):
    df[col]= df[col].str.extract(r'(\d+)', expand=False)
    
    return df[col]

In [8]:
df['Bottlenose Dolphin'] = extracting_numbers(df,'Bottlenose Dolphin')
df['Common Dolphin'] = extracting_numbers(df,'Common Dolphin')
df['Pacific White sided Dolphin'] = extracting_numbers(df,'Pacific White sided Dolphin')
df['Rissos Dolphin'] = extracting_numbers(df,'Rissos Dolphin')
df['Whales'] = extracting_numbers(df,'Whales')
df['Gray Whales'] = extracting_numbers(df,'Gray Whales')
df['Number of trips'] = extracting_numbers(df,'Number of trips')


#### 5. Getting rid of NAN values

In [9]:
## NAN values --> 0 (except in 'Number of trips')

for col in df:
    if col != 'Number of trips':
        df[col].replace(np.nan, 0, inplace=True)


#### 6. Removing redundent rows

In [10]:
## Removing rows without trips

df = df[~df['Number of trips'].isin(['Inclement Weather', '0',
                                     'INCLEMENT WEATHER', 'NO TRIPS', 'No Trips',
                                     'Happy Thanksgiving!', 'Happy Easter!',
                                     'Merry Christmas!' ])]
df = df[~df['Date'].isin(['Happy New Year!','0',0])]


#### 7. Changing 'Other' column to be Binary

In [11]:
## Other --> Binary 
##  0  =  No special sightings
##  1  =  At least one special sighting

Other = df['Other'].tolist()

for i in range (df.shape[0]):
    if Other[i] != 0 and Other[i] != '0':
        if 'No Sightings' in str(Other[i]):
            Other[i] = 0
            
        else:
            Other[i] = 1

df['Other'] = Other

#### 8. Fixing the Date format 

In [12]:
## A function that returns a column with numbers only

def extracting_string(df,col):
    df[col]= df[col].str.extract(r'(\w+)', expand=False)
    
    return df[col]

In [13]:
# Fixing the Date to be in the right format

df_copy = df.copy()
df_copy2 = df.copy()
Days = extracting_numbers(df_copy,'Date')
Months = extracting_string(df_copy2,'Date')
Days = Days.tolist()
Months=Months.tolist()


In [14]:
Years = [0]*len(Months)
j = 2
for i in range (len(Months)):
    
    if 'Dec' in str(Months[i]):
        Months[i] = '12'
    if 'Nov' in str(Months[i]):
        Months[i] = '11'
    if 'Oc' in str(Months[i]):
        Months[i] = '10'
    if 'Sep' in str(Months[i]):
        Months[i] = '09'
    if 'Aug' in str(Months[i]):
        Months[i] = '08'
    if 'Jul' in str(Months[i]):
        Months[i] = '07'
    if 'Jun' in str(Months[i]):
        Months[i] = '06'
    if 'May' in str(Months[i]).capitalize():
        Months[i] = '05'
    if 'Apr' in str(Months[i]):
        Months[i] = '04'
    if 'Mar' in str(Months[i]):
        Months[i] = '03'
    if 'Feb' in str(Months[i]):
        Months[i] = '02'
    if 'Jan' in str(Months[i]):
        Months[i] = '01'

    if Months[i] == '12' and Months[i-1] == '01':
        j += 1
    if j < 10:
        Years[i] = '200' + str(j) 
    else:
        Years[i] = '20' + str(j)  

In [15]:
Dates = []
for i in range (df.shape[0]):
    if int(Days[i]) < 10:
        Dates.append(Years[i] + '-' + Months[i] + '-' + '0' + Days[i])
    else:
        Dates.append(Years[i] + '-' + Months[i] + '-' + Days[i])

In [16]:
df['Date'] = Dates
df['Day']=Days
df['Months']=Months
df['Year']=Years

#### 9. Casting all relevant cells to int or float

In [17]:

df['Number of trips'] = pd.to_numeric(df['Number of trips'])
df['Whales'] = df['Whales'].astype(int)
df['Bottlenose Dolphin'] = df['Bottlenose Dolphin'].astype(int)
df['Gray Whales'] = df['Gray Whales'].astype(int)
df['Pacific White sided Dolphin'] = df['Pacific White sided Dolphin'].astype(int)   
df["Rissos Dolphin"] = df["Rissos Dolphin"].astype(int)
df["Common Dolphin"] = df["Common Dolphin"].astype(int)
df["Other"] = df["Other"].astype(int)
df["Day"] = df["Day"].astype(int)
df["Months"] = df["Months"].astype(int)
df["Year"] = df["Year"].astype(int)


## Adding the Weather data to our Dataframe

In [18]:
weather_df = pd.read_csv("San Diego Weather - GHCN Daily.csv")
weather_df = weather_df.drop(weather_df[weather_df.Date < '2002'].index)
weather_df = weather_df.reset_index(drop=True)


In [19]:
dates_sights=list(df['Date'])
temp_list=list(weather_df['Avg Temp (C)'])
wind_list=list(weather_df['Average Wind Speed (m/s)'])
dates_weather=list(weather_df['Date'])
for day in dates_sights:
    if day not in dates_weather:
        dates_weather.append(day)
        temp_list.append(np.nan)
        wind_list.append(np.nan)


In [20]:
weather_df = pd.DataFrame({'Date':dates_weather,'Avg Temp (C)':temp_list,'Avg Wind Speed (M/S)':wind_list})

## Merging

In [21]:
merged_df = pd.merge(df, weather_df, on = 'Date')

####  1. Handling outliers

In [22]:
## Outlier --> NAN

merged_df.loc[merged_df['Avg Wind Speed (M/S)']==-999.9,'Avg Wind Speed (M/S)']= np.nan
merged_df.loc[merged_df['Avg Temp (C)']==-999.9,'Avg Temp (C)']= np.nan

## Dolphins live in packs
## Outlier for Dolphins will be the value 1

merged_df['Common Dolphin'][merged_df['Common Dolphin']==1] = merged_df['Common Dolphin'].mean()
merged_df["Common Dolphin"] = merged_df["Common Dolphin"].astype(int)


merged_df['Pacific White sided Dolphin'][merged_df['Pacific White sided Dolphin']==1] = merged_df['Pacific White sided Dolphin'].mean()
merged_df["Pacific White sided Dolphin"] = merged_df["Pacific White sided Dolphin"].astype(int)


merged_df['Rissos Dolphin'][merged_df['Rissos Dolphin']==1] = merged_df['Rissos Dolphin'].mean()
merged_df["Rissos Dolphin"] = merged_df["Rissos Dolphin"].astype(int)


#### 2. Filling Missing values with mean values based on months

In [23]:
merged_df['Months'] = Months

In [24]:
def Filling_Missing_Values(df, col):
    g = df.groupby('Months')   ##  Making a group of every month
    monthly_averages = g.aggregate({col:np.mean})  ##  Getting an average of the given col based on every grouped month
    mon = list()  

    for i in range(12):
        mon.append(monthly_averages[col][i])  ##  A list of Averages
  
    for i, row in df.iterrows(): 
        
        if pd.isnull(row[col]):  
      
            if df['Months'].loc[i] == '01':
                df[col].loc[i]= mon[0]
            
            if df['Months'].loc[i] == '02':
                df[col].loc[i]= mon[1]
            
            if df['Months'].loc[i] == '03':
                df[col].loc[i]= mon[2]
            
            if df['Months'].loc[i] == '04':
                df[col].loc[i]= mon[3]
            
            if df['Months'].loc[i] == '05':
                df[col].loc[i]= mon[4]
            
            if df['Months'].loc[i] == '06':
                df[col].loc[i]= mon[5]
            
            if df['Months'].loc[i] == '07':
                df[col].loc[i]= mon[6]
            
            if df['Months'].loc[i] == '08':
                df[col].loc[i]= mon[7]
            
            if df['Months'].loc[i] == '09':
                df[col].loc[i]= mon[8]
            
            if df['Months'].loc[i] == '10':
                df[col].loc[i]= mon[9]
            
            if df['Months'].loc[i] == '11':
                df[col].loc[i]= mon[10]
            
            if df['Months'].loc[i] == '12':
                df[col].loc[i]= mon[11]
               


In [25]:
Filling_Missing_Values(merged_df, 'Avg Temp (C)')
Filling_Missing_Values(merged_df, 'Avg Wind Speed (M/S)')
Filling_Missing_Values(merged_df, 'Number of trips')

merged_df['Number of trips'] = merged_df['Number of trips'].astype(int) ## Number of trips cannot be float


####  Adding 'Seasons' column

In [26]:
##  Winter = 1
##  Spring = 2
##  Summer = 3
##  Fall = 4

Season = [0]*(merged_df.shape[0]) 

for i, cell in enumerate (merged_df['Months']):
    if cell =='12' or cell == '01' or cell == '02':  
        Season[i] = 1
    if cell =='03' or cell == '04' or cell == '05':  
        Season[i] = 2
    if cell =='06' or cell == '07' or cell == '08':
        Season[i] = 3
    if cell =='09' or cell == '10' or cell == '11':
        Season[i] = 4
        
merged_df['Seasons'] = Season

In [27]:
merged_df.to_csv('FINAL_DF.csv')
merged_df

,Date,Number of trips,Bottlenose Dolphin,Common Dolphin,Pacific White sided Dolphin,Rissos Dolphin,Whales,Gray Whales,Other,Day,Months,Year,Avg Temp (C),Avg Wind Speed (M/S),Seasons
0,2002-12-31,2,0,20,30,0,0,0,0,31,12,2002,12.750000,2.200000,1
1,2002-12-30,2,0,795,20,0,2,2,0,30,12,2002,12.750000,1.300000,1
2,2002-12-28,2,0,795,4,0,0,1,0,28,12,2002,12.750000,1.900000,1
3,2002-12-27,2,0,795,4,0,0,0,0,27,12,2002,13.900000,1.800000,1
4,2002-12-26,2,0,795,0,0,1,1,0,26,12,2002,11.700000,1.600000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,2021-02-05,2,0,1525,0,0,3,3,0,5,02,2021,14.915158,2.243991,1
6508,2021-02-04,1,2,0,50,0,0,0,0,4,02,2021,14.915158,2.243991,1
6509,2021-02-03,1,0,300,35,0,0,0,0,3,02,2021,14.915158,2.243991,1
6510,2021-02-02,1,0,0,0,0,3,3,0,2,02,2021,14.915158,2.243991,1


### Binary Data for the Machine learning

In [28]:
Binary_Data = merged_df.copy()
Binary_Data['Common Dolphin'][Binary_Data['Common Dolphin']>  0] = 1
Binary_Data['Gray Whales'][Binary_Data['Gray Whales'] > 0] = 1
Binary_Data['Whales'][Binary_Data['Whales']>  0] = 1
Binary_Data['Pacific White sided Dolphin'][Binary_Data['Pacific White sided Dolphin'] > 0] = 1
Binary_Data['Rissos Dolphin'][Binary_Data['Rissos Dolphin']>  0] = 1
Binary_Data['Bottlenose Dolphin'][Binary_Data['Bottlenose Dolphin']>  0] = 1


In [29]:
Binary_Data.to_csv("Binary_FINAL_DF.csv")
Binary_Data

,Date,Number of trips,Bottlenose Dolphin,Common Dolphin,Pacific White sided Dolphin,Rissos Dolphin,Whales,Gray Whales,Other,Day,Months,Year,Avg Temp (C),Avg Wind Speed (M/S),Seasons
0,2002-12-31,2,0,1,1,0,0,0,0,31,12,2002,12.750000,2.200000,1
1,2002-12-30,2,0,1,1,0,1,1,0,30,12,2002,12.750000,1.300000,1
2,2002-12-28,2,0,1,1,0,0,1,0,28,12,2002,12.750000,1.900000,1
3,2002-12-27,2,0,1,1,0,0,0,0,27,12,2002,13.900000,1.800000,1
4,2002-12-26,2,0,1,0,0,1,1,0,26,12,2002,11.700000,1.600000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,2021-02-05,2,0,1,0,0,1,1,0,5,02,2021,14.915158,2.243991,1
6508,2021-02-04,1,1,0,1,0,0,0,0,4,02,2021,14.915158,2.243991,1
6509,2021-02-03,1,0,1,1,0,0,0,0,3,02,2021,14.915158,2.243991,1
6510,2021-02-02,1,0,0,0,0,1,1,0,2,02,2021,14.915158,2.243991,1
